# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [1]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [2]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

            # Note: This logic does not properly work
#         elif order.status in [order.Expired]:
#             if order.isbuy():
#                 self.log('BUY ORDER EXPIRED')
#             elif order.issell():
#                 self.log('SELL ORDER EXPIRED')
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None



    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            # TODO: Make this a filter in the class constructor
            if (data.prediction[0] >= data.close[0]):
                self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=self.size) # market order buys at next open
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            if data.prediction[0] > data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')
                # Keep track of the created order to avoid a 2nd order, order valid for 1 day
                self.order = self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
            # if prediction is less than current value sell at open
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.order = self.sell(size=self.size)
        

In [3]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [4]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 


## Import Data and Run Backtest

In [5]:
import os

In [6]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../data/ticker_predictions/{pred_folder_name}'):
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df['prediction'] = pred_df[pred_col_name]
        # remove all non-prediction rows, use "timestamp" as the common datetime name
        ticker_df = ticker_df[ticker_df.prediction > 0]
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)

        # make timestamp in column position 0 
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']

        # skip nonfeatures, keep timestamp
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]

        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict
#     return ticker_df

In [7]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('linreg_preds', 'prediction', '../data/ticker_data')
cerebro_dict

--------Ticker Name-------: UNH
BUY AND HOLD
Starting Portfolio Value: 100000
2000-04-03, BUY EXECUTED, 7.42
Stop price: 22.700000762939453
ROI:        205.84%
Final Portfolio Value: 305842.18840408325

STRATEGY INFO:
OrderedDict([('sharperatio', 0.6276447431375926)])
AutoOrderedDict([('len', 1), ('drawdown', 0.41674815646385877), ('moneydown', 1279.9257488250732), ('max', AutoOrderedDict([('len', 79), ('drawdown', 19.84538234303845), ('moneydown', 44225.138946533174)]))])
OrderedDict([('rtot', 1.117899058763177), ('ravg', 0.00206254438886195), ('rnorm', 0.6816260058973242), ('rnorm100', 68.16260058973242)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2000-03-31, Current Portfolio Value : 100000.0
2000-03-31, MARKET BUY CREATE 13473 shares at next open, current close price: 7.453125
2000-04-03, BUY EXECUTED, 7.42
2000-04-03, Current Portfolio Value : 104526.09236192703
2000-04-03, LIMIT SELL CREATE 13473 shares at 241.0939114132067
2000-

2000-09-22, Current Portfolio Value : 164733.56753635406
2000-09-22, LIMIT SELL CREATE 13473 shares at 301.8592733172036
2000-09-25, Current Portfolio Value : 163891.50503635406
2000-09-25, LIMIT SELL CREATE 13473 shares at 299.7230854089225
2000-09-26, Current Portfolio Value : 164102.02066135406
2000-09-26, LIMIT SELL CREATE 13473 shares at 274.1199484274955
2000-09-27, Current Portfolio Value : 165049.328125
2000-09-27, LIMIT SELL CREATE 13473 shares at 307.2556994965281
2000-09-28, Current Portfolio Value : 167470.27066135406
2000-09-28, LIMIT SELL CREATE 13473 shares at 261.1253839932098
2000-09-29, Current Portfolio Value : 166312.421875
2000-09-29, LIMIT SELL CREATE 13473 shares at 276.630157613752
2000-10-02, Current Portfolio Value : 178417.08316135406
2000-10-02, LIMIT SELL CREATE 13473 shares at 258.5640075472894
2000-10-03, Current Portfolio Value : 177364.50503635406
2000-10-03, LIMIT SELL CREATE 13473 shares at 251.4563934995004
2000-10-04, Current Portfolio Value : 17273

2001-03-21, Current Portfolio Value : 182901.0505552292
2001-03-21, LIMIT SELL CREATE 13473 shares at 337.6284931924451
2001-03-22, Current Portfolio Value : 177511.8556947708
2001-03-22, LIMIT SELL CREATE 13473 shares at 333.08956394148663
2001-03-23, Current Portfolio Value : 183574.703125
2001-03-23, LIMIT SELL CREATE 13473 shares at 330.1951896945299
2001-03-26, Current Portfolio Value : 188963.89798545834
2001-03-26, LIMIT SELL CREATE 13473 shares at 324.3439918199844
2001-03-27, Current Portfolio Value : 189772.28363895416
2001-03-27, LIMIT SELL CREATE 13473 shares at 329.83652097281424
2001-03-28, Current Portfolio Value : 188963.89798545834
2001-03-28, LIMIT SELL CREATE 13473 shares at 322.67435306435783
2001-03-29, Current Portfolio Value : 194656.24870872498
2001-03-29, LIMIT SELL CREATE 13473 shares at 320.2063003738604
2001-03-30, Current Portfolio Value : 199607.5674715042
2001-03-30, LIMIT SELL CREATE 13473 shares at 328.4781169061718
2001-04-02, Current Portfolio Value :

2001-09-20, Current Portfolio Value : 206849.31011104584
2001-09-20, LIMIT SELL CREATE 13473 shares at 345.2451644659387
2001-09-21, Current Portfolio Value : 201931.6681947708
2001-09-21, LIMIT SELL CREATE 13473 shares at 343.17156412410304
2001-09-24, Current Portfolio Value : 207522.96268081665
2001-09-24, LIMIT SELL CREATE 13473 shares at 345.0071257218696
2001-09-25, Current Portfolio Value : 212171.1512784958
2001-09-25, LIMIT SELL CREATE 13473 shares at 363.55276898568326
2001-09-26, Current Portfolio Value : 220962.27298545837
2001-09-26, LIMIT SELL CREATE 13473 shares at 362.4419261254693
2001-09-27, Current Portfolio Value : 222275.88842964172
2001-09-27, LIMIT SELL CREATE 13473 shares at 363.1170803595419
2001-09-28, Current Portfolio Value : 223993.703125
2001-09-28, LIMIT SELL CREATE 13473 shares at 358.30255725728347
2001-10-01, Current Portfolio Value : 225341.00826454163
2001-10-01, LIMIT SELL CREATE 13473 shares at 362.5377344277332
2001-10-02, Current Portfolio Value 

2002-03-13, Current Portfolio Value : 248076.69576454163
2002-03-13, LIMIT SELL CREATE 13473 shares at 448.3173871158537
2002-03-14, Current Portfolio Value : 250602.88326454163
2002-03-14, LIMIT SELL CREATE 13473 shares at 441.3893566693753
2002-03-15, Current Portfolio Value : 251108.12590408325
2002-03-15, LIMIT SELL CREATE 13473 shares at 435.9597960772666
2002-03-18, Current Portfolio Value : 252388.05165290833
2002-03-18, LIMIT SELL CREATE 13473 shares at 434.2316323324982
2002-03-19, Current Portfolio Value : 253667.97740173346
2002-03-19, LIMIT SELL CREATE 13473 shares at 430.14257211090217
2002-03-20, Current Portfolio Value : 253634.31340408325
2002-03-20, LIMIT SELL CREATE 13473 shares at 427.78558690304
2002-03-21, Current Portfolio Value : 253769.0207901001
2002-03-21, LIMIT SELL CREATE 13473 shares at 431.22976986535934
2002-03-22, Current Portfolio Value : 251545.98915290833
2002-03-22, LIMIT SELL CREATE 13473 shares at 429.1488551119627
2002-03-25, Current Portfolio Val

...

2001-03-06, Current Portfolio Value : 62952.5
2001-03-06, LIMIT SELL CREATE 2117 shares at 214.12660212687356
2001-03-07, Current Portfolio Value : 64275.625
2001-03-07, LIMIT SELL CREATE 2117 shares at 212.42418283940765
2001-03-08, Current Portfolio Value : 62754.03125
2001-03-08, LIMIT SELL CREATE 2117 shares at 217.0644201183528
2001-03-09, Current Portfolio Value : 60041.625
2001-03-09, LIMIT SELL CREATE 2117 shares at 216.1499013930942
2001-03-12, Current Portfolio Value : 55013.75
2001-03-12, LIMIT SELL CREATE 2117 shares at 215.0821944024323
2001-03-13, Current Portfolio Value : 57395.375
2001-03-13, LIMIT SELL CREATE 2117 shares at 217.2025681901771
2001-03-14, Current Portfolio Value : 57196.90625
2001-03-14, LIMIT SELL CREATE 2117 shares at 223.56196114664408
2001-03-15, Current Portfolio Value : 56866.125
2001-03-15, LIMIT SELL CREATE 2117 shares at 233.5285710557589
2001-03-16, Current Portfolio Value : 57792.3125
2001-03-16, LIMIT SELL CREATE 2117 shares at 230.2889301341

2001-07-26, Current Portfolio Value : 70523.41737365723
2001-07-26, LIMIT SELL CREATE 2117 shares at 225.5089083985295
2001-07-27, Current Portfolio Value : 69337.90254211426
2001-07-27, LIMIT SELL CREATE 2117 shares at 221.2347403098928
2001-07-30, Current Portfolio Value : 69687.20948028564
2001-07-30, LIMIT SELL CREATE 2117 shares at 218.8815065648717
2001-07-31, Current Portfolio Value : 70100.02383422852
2001-07-31, LIMIT SELL CREATE 2117 shares at 219.7267254177132
2001-08-01, Current Portfolio Value : 70396.40254211426
2001-08-01, LIMIT SELL CREATE 2117 shares at 219.29444997025644
2001-08-02, Current Portfolio Value : 71433.72801971436
2001-08-02, LIMIT SELL CREATE 2117 shares at 217.16701242981472
2001-08-03, Current Portfolio Value : 70840.97060394287
2001-08-03, LIMIT SELL CREATE 2117 shares at 221.6617564773585
2001-08-06, Current Portfolio Value : 70036.50834274292
2001-08-06, LIMIT SELL CREATE 2117 shares at 232.19957074584025
2001-08-07, Current Portfolio Value : 70269.3

2001-12-18, Current Portfolio Value : 73360.19769668579
2001-12-18, LIMIT SELL CREATE 2117 shares at 251.41811396130348
2001-12-19, Current Portfolio Value : 73593.06898880005
2001-12-19, LIMIT SELL CREATE 2117 shares at 250.53923446220784
2001-12-20, Current Portfolio Value : 70703.36851119995
2001-12-20, LIMIT SELL CREATE 2117 shares at 249.47160495625548
2001-12-21, Current Portfolio Value : 71528.9972190857
2001-12-21, LIMIT SELL CREATE 2117 shares at 246.60529789024537
2001-12-24, Current Portfolio Value : 71243.19769668579
2001-12-24, LIMIT SELL CREATE 2117 shares at 255.04944667113955
2001-12-26, Current Portfolio Value : 71677.18657302856
2001-12-26, LIMIT SELL CREATE 2117 shares at 256.67665587601863
2001-12-27, Current Portfolio Value : 71857.12963485718
2001-12-27, LIMIT SELL CREATE 2117 shares at 256.3241918715873
2001-12-28, Current Portfolio Value : 71878.30415725708
2001-12-28, LIMIT SELL CREATE 2117 shares at 255.55325197044272
2001-12-31, Current Portfolio Value : 7016

2002-05-29, Current Portfolio Value : 55132.830442428596
2002-05-29, LIMIT SELL CREATE 2117 shares at 321.9290896072408
2002-05-30, Current Portfolio Value : 55757.34560394287
2002-05-30, LIMIT SELL CREATE 2117 shares at 323.25690524911454
2002-05-31, Current Portfolio Value : 53926.14108848571
2002-05-31, LIMIT SELL CREATE 2117 shares at 325.6605251032801
Final Portfolio Value: 53926.14108848571

STRATEGY INFO:
OrderedDict([('sharperatio', -0.08149073316471005)])
AutoOrderedDict([('len', 541), ('drawdown', 46.07385891151429), ('moneydown', 46073.85891151429), ('max', AutoOrderedDict([('len', 541), ('drawdown', 56.03434375), ('moneydown', 56034.34375)]))])
OrderedDict([('rtot', -0.6175548332786981), ('ravg', -0.001139400061399812), ('rnorm', -0.24958494245957547), ('rnorm100', -24.958494245957546)])
For MSFT, return was -46.073858911514286% for buy and hold vs -46.073858911514286%  for strategy








{'UNH': <backtrader.cerebro.Cerebro at 0x7fdaf8954310>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7fdab8b8ab50>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7fdae89ce250>,
 'WAT': <backtrader.cerebro.Cerebro at 0x7fdac9264c90>,
 'HD': <backtrader.cerebro.Cerebro at 0x7fdaf8c53590>,
 'MSFT': <backtrader.cerebro.Cerebro at 0x7fdac8a9dc90>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [22]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/anaconda3/lib/python3.7/site-packages/matplotlib/dates.py)

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  